In [ ]:
# Naive Bayes binary classifier on Boolean features

from typing import List, Set

class BooleanFeature:
    def __init__(self, name: str, value: bool):
        self.name = name
        self.value = value

    def __eq__(self, other):
        return self.name == other.name and self.value == other.value

    def __hash__(self):
        return hash((self.name, self.value))

class BooleanLabel(BooleanFeature):
    pass

class BooleanInstance:
    def __init__(self, features: Set[BooleanFeature], label: BooleanLabel):
        self.features = features
        self.label = label

class NaiveBayesModel:
    def __init__(self, instances: List[BooleanInstance]):
        self.instances = instances
        self.true_instances = [i for i in instances if i.label.value]
        self.probability_true = len(self.true_instances) / len(instances)
        
        # all feature names
        self.feature_names = set(f.name for i in instances for f in i.features)
        
        # Calculate P(feature=True | label=True) for each feature
        self.feature_probabilities = {}
        for fname in self.feature_names:
            count_true = sum(
                1 for inst in self.true_instances 
                for f in inst.features if f.name == fname and f.value
            )
            self.feature_probabilities[fname] = count_true / len(self.true_instances) if self.true_instances else 0.0

        # Naive Bayes numerator = P(label=True) * product of P(features | label=True)
        self.numerator = self.probability_true
        for prob in self.feature_probabilities.values():
            self.numerator *= prob

    def probability_feature_vector(self, features: Set[BooleanFeature]) -> float:
        matching_count = sum(1 for i in self.instances if i.features == features)
        return matching_count / len(self.instances)

    def predict(self, features: Set[BooleanFeature]) -> float:
        # Return posterior proportional to numerator / P(features)
        denominator = self.probability_feature_vector(features)
        if denominator == 0:
            return 0.0
        return self.numerator / denominator

# Example usage
features1 = {BooleanFeature("high_absences", True), BooleanFeature("passed", True)}
label1 = BooleanLabel("passed", True)

features2 = {BooleanFeature("high_absences", False), BooleanFeature("passed", False)}
label2 = BooleanLabel("passed", False)

instances = [BooleanInstance(features1, label1), BooleanInstance(features2, label2)]

model = NaiveBayesModel(instances)

test_features = {BooleanFeature("high_absences", True), BooleanFeature("passed", True)}
print("Prediction score:", model.predict(test_features))


In [ ]:
import Pyro4
import concurrent.futures
import time

class JobConfiguration:
    def __init__(self, content_path, style_path, model_path, model_type,
                 width=800, alpha=1.0, beta=200.0, iterations=5000):
        self.content_path = content_path
        self.style_path = style_path
        self.model_path = model_path
        self.model_type = model_type
        self.width = width
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

def call_server(remote_server, job_config):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_call = executor.submit(
            remote_server.generate,
            job_config.content_path,
            job_config.style_path,
            job_config.model_path,
            job_config.model_type,
            job_config.width,
            job_config.alpha,
            job_config.beta,
            job_config.iterations
        )
        try:
            result = future_call.result(timeout=3600)  # 1 hour timeout
            return result
        except concurrent.futures.TimeoutError:
            return False

if __name__ == "__main__":
    ns = Pyro4.locateNS()
    remote_server = Pyro4.Proxy(ns.lookup("neuralserver"))

    job_config = JobConfiguration(
        "/path/to/content.jpg",
        "/path/to/style.jpg",
        "/path/to/model",
        "VGG",
        iterations=1000
    )

    success = call_server(remote_server, job_config)
    print("Style transfer success:", success)

    remote_server._pyroRelease()


In [ ]:
# spark_student_performance_pipeline.ipynb

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors

spark = SparkSession.builder.appName("StudentPerformancePipeline").master("local[*]").getOrCreate()

# Load data (using local CSV or UCI direct download)
data_path = "student-mat.csv"  # Download and put in working dir or update path
df = spark.read.csv(data_path, header=True, sep=";")

# Quick preprocessing: cast label to int (e.g., passed = G3 >= 10)
from pyspark.sql.functions import col, when

df = df.withColumn("G3_int", col("G3").cast("int"))
df = df.withColumn("passed", when(col("G3_int") >= 10, 1).otherwise(0))

# Select features - for simplicity, pick numeric features only and assemble vector
from pyspark.ml.feature import VectorAssembler

feature_cols = ["absences", "age", "failures", "G1", "G2"]
for c in feature_cols:
    df = df.withColumn(c, col(c).cast("double"))

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

data = assembler.transform(df).select("features", "passed").withColumnRenamed("passed", "label")

# Index label and features
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

(trainingData, testData) = data.randomSplit([0.8, 0.2])

# Decision Tree classifier pipeline
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt, labelConverter])

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

predictions.select("predictedLabel", "label", "features").show(5, truncate=False)

# Random Forest classifier pipeline
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

pipeline_rf = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])
model_rf = pipeline_rf.fit(trainingData)

predictions_rf = model_rf.transform(testData)

predictions_rf.select("predictedLabel", "label", "features").show(5, truncate=False)

spark.stop()

